<a href="https://colab.research.google.com/github/AichaELMouta/Car_Price_Prediction/blob/main/Copy_of_Car_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zafarali27/car-price-prediction")

print("Path to dataset files:", path)


Using Colab cache for faster access to the 'car-price-prediction' dataset.
Path to dataset files: /kaggle/input/car-price-prediction


In [1]:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
#import all regression models to comparing metric values
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn import metrics

Loading Dataset from csv file to pandas dataframe

In [4]:
car_data=pd.read_csv("/content/car_price_prediction_.csv")

In [12]:
#Inspecting data
car_data.head()

,Car ID,Brand,Year,Engine Size,Fuel Type,Transmission,Mileage,Condition,Price,Model
0,1,Tesla,2016,2.3,Petrol,Manual,114832,New,26613.92,Model X
1,2,BMW,2018,4.4,Electric,Manual,143190,Used,14679.61,5 Series
2,3,Audi,2013,4.5,Electric,Manual,181601,New,44402.61,A4
3,4,Tesla,2011,4.1,Diesel,Automatic,68682,New,86374.33,Model Y
4,5,Ford,2009,2.6,Diesel,Manual,223009,Like New,73577.10,Mustang


In [10]:
car_data.shape

(2500, 10)

In [11]:
car_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Car ID        2500 non-null   int64  
 1   Brand         2500 non-null   object 
 2   Year          2500 non-null   int64  
 3   Engine Size   2500 non-null   float64
 4   Fuel Type     2500 non-null   object 
 5   Transmission  2500 non-null   object 
 6   Mileage       2500 non-null   int64  
 7   Condition     2500 non-null   object 
 8   Price         2500 non-null   float64
 9   Model         2500 non-null   object 
dtypes: float64(2), int64(3), object(5)
memory usage: 195.4+ KB


In [13]:
car_data.isnull().sum()

,0
Car ID,0
Brand,0
Year,0
Engine Size,0
Fuel Type,0
Transmission,0
Mileage,0
Condition,0
Price,0
Model,0


In [16]:
car_data.describe()

,Car ID,Year,Engine Size,Mileage,Price
count,2500.00000,2500.0000,2500.000000,2500.000000,2500.000000
mean,1250.50000,2011.6268,3.465240,149749.844800,52638.022532
std,721.83216,6.9917,1.432053,87919.952034,27295.833455
min,1.00000,2000.0000,1.000000,15.000000,5011.270000
25%,625.75000,2005.0000,2.200000,71831.500000,28908.485000
50%,1250.50000,2012.0000,3.400000,149085.000000,53485.240000
75%,1875.25000,2018.0000,4.700000,225990.500000,75838.532500
max,2500.00000,2023.0000,6.000000,299967.000000,99982.590000


In [17]:
car_data.duplicated().sum()

np.int64(0)

In [29]:
car_data.rename(columns={"Fuel Type":"Fuel_Type"}, inplace=True)

In [40]:
car_data.rename(columns={"Engine Size":"Engine_size"}, inplace=True)

In [41]:
car_data.columns

Index(['Car ID', 'Brand', 'Year', 'Engine_size', 'Fuel_Type', 'Transmission',
       'Mileage', 'Condition', 'Price', 'Model'],
      dtype='object')

In [43]:
car_data.Brand.value_counts()

,count
Brand,
Toyota,374
Audi,368
BMW,358
Mercedes,353
Honda,352
Tesla,348
Ford,347


In [45]:
#checking the categorical data
print(car_data.Fuel_Type.value_counts())
print(car_data.Transmission.value_counts())
print(car_data.Condition.value_counts())
print(car_data.Brand.value_counts())

Fuel_Type
Diesel      655
Petrol      630
Electric    614
Hybrid      601
Name: count, dtype: int64
Transmission
Manual       1308
Automatic    1192
Name: count, dtype: int64
Condition
Used        855
Like New    836
New         809
Name: count, dtype: int64
Brand
Toyota      374
Audi        368
BMW         358
Mercedes    353
Honda       352
Tesla       348
Ford        347
Name: count, dtype: int64


Encoding the Categorical Data

In [48]:
#Encoding Fuel Type
car_data.replace({'Fuel_Type':{'Petrol':0, 'Diesel':2, 'Electric':3, 'Hybrid':4}}, inplace=True)

In [47]:
#Encoding Transmission
car_data.replace({'Transmission':{'Manual':0, 'Automatic':1}}, inplace=True)

/tmp/ipython-input-2395921181.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  car_data.replace({'Transmission':{'Manual':0, 'Automatic':1}}, inplace=True)


In [51]:
#Encoding Condition
car_data.replace({'Condition':{'Like New':0, 'New':1, 'Used':2}}, inplace=True)

/tmp/ipython-input-1115319375.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  car_data.replace({'Condition':{'Like New':0, 'New':1, 'Used':2}}, inplace=True)


In [52]:
#Encoding Brand
car_data.replace({'Brand':{'Toyota':0, 'Audi':1, 'BMW':2, 'Mercedes':3, 'Honda':4, 'Tesla':5, 'Ford':6}}, inplace=True)

/tmp/ipython-input-2207127047.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  car_data.replace({'Brand':{'Toyota':0, 'Audi':1, 'BMW':2, 'Mercedes':3, 'Honda':4, 'Tesla':5, 'Ford':6}}, inplace=True)


In [53]:
car_data.head()

,Car ID,Brand,Year,Engine_size,Fuel_Type,Transmission,Mileage,Condition,Price,Model
0,1,5,2016,2.3,0,0,114832,1,26613.92,Model X
1,2,2,2018,4.4,3,0,143190,2,14679.61,5 Series
2,3,1,2013,4.5,3,0,181601,1,44402.61,A4
3,4,5,2011,4.1,2,1,68682,1,86374.33,Model Y
4,5,6,2009,2.6,2,0,223009,0,73577.10,Mustang


Data Analysis and Visualization